In [1]:
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, Subset
import torch.optim as optim

Load CIFAR10 train and test set

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Subset the CIFAR10 dataset
classes_of_interest = [3, 5, 8]  # indices for cat, dog, and ship
subset_train_indices = [i for i in range(len(trainset)) if trainset.targets[i] in classes_of_interest][:3000]
subset_test_indices = [i for i in range(len(testset)) if testset.targets[i] in classes_of_interest][:1000]

train_subset = Subset(trainset, subset_train_indices)
test_subset = Subset(testset, subset_test_indices)

# Initialize data loaders
trainloader = DataLoader(train_subset, batch_size=4, shuffle=True)
testloader = DataLoader(test_subset, batch_size=4, shuffle=False)

In [4]:
class SimpleFCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleFCNN, self).__init__()
        self.flatten = nn.Flatten()  # Flatten the input images

        self.hidden = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.flatten(x)
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return x

input_size = 3 * 32 * 32 
hidden_size = 512
output_size = len(classes_of_interest)  # Number of classes (cat, dog, ship)

simple_nn = SimpleFCNN(input_size, hidden_size, output_size)

In [5]:
optimizer = optim.SGD(simple_nn.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
epochs = 10

for epoch in range(epochs):
    # Training
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    best_accuracy = 0.0

    for inputs, labels in trainloader:
        optimizer.zero_grad()

        outputs = simple_nn(inputs)
        labels = torch.tensor([classes_of_interest.index(label.item()) for label in labels])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = correct_train / total_train

    # Testing
    simple_nn.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = simple_nn(inputs)
            labels = torch.tensor([classes_of_interest.index(label.item()) for label in labels])

            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_accuracy = correct_test / total_test

    # Print and save the best model
    print(f'Epoch {epoch + 1}/{epochs}: Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')


    if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            torch.save(simple_nn.state_dict(), 'best_model.pth')

Epoch 1/10: Train Accuracy: 0.6057, Test Accuracy: 0.6770
Epoch 2/10: Train Accuracy: 0.6857, Test Accuracy: 0.6450
Epoch 3/10: Train Accuracy: 0.7107, Test Accuracy: 0.6540
Epoch 4/10: Train Accuracy: 0.7463, Test Accuracy: 0.6600
Epoch 5/10: Train Accuracy: 0.7677, Test Accuracy: 0.6560
Epoch 6/10: Train Accuracy: 0.8083, Test Accuracy: 0.6530
Epoch 7/10: Train Accuracy: 0.8307, Test Accuracy: 0.6710
Epoch 8/10: Train Accuracy: 0.8463, Test Accuracy: 0.6540
Epoch 9/10: Train Accuracy: 0.8797, Test Accuracy: 0.6500
Epoch 10/10: Train Accuracy: 0.8787, Test Accuracy: 0.6790


In [6]:
class_names = ['cat', 'dog', 'ship']

best_model = SimpleFCNN(input_size, hidden_size, output_size)
best_model.load_state_dict(torch.load('best_model.pth'))
best_model.eval()

correct_overall = 0
total_overall = 0
correct_per_class = [0] * len(class_names)
total_per_class = [0] * len(class_names)

with torch.no_grad():
    for inputs, labels in testloader:
        outputs = best_model(inputs)
        labels = torch.tensor([classes_of_interest.index(label.item()) for label in labels])

        _, predicted = torch.max(outputs.data, 1)

        total_overall += labels.size(0)
        correct_overall += (predicted == labels).sum().item()

        for i in range(len(class_names)):
            class_indices = (labels == i).nonzero()
            total_per_class[i] += class_indices.size(0)
            correct_per_class[i] += (predicted[class_indices] == i).sum().item()

overall_accuracy = correct_overall / total_overall
per_class_accuracy = [correct_per_class[i] / total_per_class[i] for i in range(len(class_names))]

best_class_index = per_class_accuracy.index(max(per_class_accuracy))
best_class_name = class_names[best_class_index]

print(f'Overall Test Accuracy: {overall_accuracy:.4f}')

for i, class_name in enumerate(class_names):
    print(f'Test Accuracy for {class_name}: {per_class_accuracy[i]:.4f}')

print(f'The classifier performs best on the class "{best_class_name}"')


Overall Test Accuracy: 0.6790
Test Accuracy for cat: 0.6220
Test Accuracy for dog: 0.5123
Test Accuracy for ship: 0.8879
The classifier performs best on the class "ship"


### SHIP PERFORMS BETTER THAN THE OTHER CLASSES, SINCE IT IS VERY DIFFERENT FROM THE OTHER TWO CLASSES. IT HAS A DISTINCT SHAPE BUT THE OTHER CLASSES, CAT AND DOG HAVE SOME STRUCTURAL SIMILARITIES BUT SHIP IS MOSTLY IN SEA AND HAVE VERY DISTINCT FEATURES THAN CAT AND DOG